## Imports

In [28]:
import torchvision.datasets as ds
from sklearn import datasets
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import pickle

## Dense Layer

In [29]:
class DenseLayer:
    def __init__(self, inputDimensions, outputDimensions):
        self.input = None
        self.output = None
        self.weights = np.random.randn(outputDimensions, inputDimensions)
        self.biases = np.zeros((outputDimensions, 1))

    def forwardPass(self, input):
        self.input = input
        self.output = np.dot(self.weights, input) + self.biases
        return self.output
    
    def backwardPass(self, dL_dout, alpha):
        dw = np.dot(dL_dout, self.input.T)
        db = np.sum(dL_dout, axis=1, keepdims=True)

        di = np.dot(self.weights.T, dL_dout)

        self.weights -= alpha * dw
        self.biases -= alpha * db

        return di

### RELU

In [30]:
class RELU:
    ## matrix that passes through relu is (n[l], m)
    def forwardPass(self, x):
        self.input = x
        return np.maximum(0, x)
    
    def backwardPass(self, x):
        return x * (self.input > 0)

### Softmax

In [31]:
class Softmax:
    def calcSoftmax(self, Y):
        #calculate from an 1D array
        exps = np.exp(Y - np.max(Y, axis=0, keepdims=True))
        return exps / np.sum(exps, axis=0, keepdims=True)
    
    def forwardPass(self, Y):
        return self.calcSoftmax(Y)
    
    def backwardPass(self, Y):
        return Y

#### Loss calc

In [32]:
def crossEntropyLoss(y, y_hat):
    return -np.sum(y * np.log(y_hat + 1e-9)) / y.shape[1]

## Loaders

In [41]:
class Loaders:
    #train a 3 hidden layer neural network
    def train_data(self, epochs, alpha, input_dimensions, output_dimensions, train_loader, dense1, activation1, dense2, activation2, dense3, activation3, dense4):
        for epoch in range(epochs):
            for images, labels in train_loader:
                total_loss = 0
                images = images.view(images.size(0), -1).numpy()
                labels_onehot = np.eye(output_dimensions)[labels.numpy()]

                #forward pass
                dense1_output = dense1.forwardPass(images.T)
                activation1_output = activation1.forwardPass(dense1_output)
                dense2_output = dense2.forwardPass(activation1_output)
                activation2_output = activation2.forwardPass(dense2_output)
                dense3_output = dense3.forwardPass(activation2_output)
                activation3_output = activation3.forwardPass(dense3_output)
                dense4_output = dense4.forwardPass(activation3_output)
                
                y_pred = Softmax().calcSoftmax(dense4_output)

                #calculate loss
                loss = crossEntropyLoss(labels_onehot, y_pred.T)
                total_loss += loss

                #backward pass
                loss_gradient = y_pred.T - labels_onehot
                dh4 = dense4.backwardPass(loss_gradient.T, alpha)
                do4 = activation3.backwardPass(dh4)
                dh3 = dense3.backwardPass(do4, alpha)
                do3 = activation2.backwardPass(dh3)
                dh2 = dense2.backwardPass(do3, alpha)
                do2 = activation1.backwardPass(dh2)
                dh1 = dense1.backwardPass(do2, alpha)

                


            print(f"Epoch: {epoch}, Loss: {loss}")

    def test_data(self, test_loader, dense1, activation1, dense2, activation2, dense3, activation3, dense4):
        
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.view(images.size(0), -1).numpy()
            labels = labels.numpy()

            #write like train
            
            dense1_output = dense1.forwardPass(images.T)
            activation1_output = activation1.forwardPass(dense1_output)
            dense2_output = dense2.forwardPass(activation1_output)
            activation2_output = activation2.forwardPass(dense2_output)
            dense3_output = dense3.forwardPass(activation2_output)
            activation3_output = activation3.forwardPass(dense3_output)
            dense4_output = dense4.forwardPass(activation3_output)
            y_pred = Softmax().calcSoftmax(dense4_output)

            predictions = np.argmax(y_pred, axis=0)
            correct += np.sum(predictions == labels)
            total += labels.shape[0]

        # print upto 6 after decimal
        print(f"Accuracy: {correct/total*100:.6f}")

    def savetoPickle(self, dense1, dense2, dense3, dense4, filename):
        weghts_and_biases = {
            "dense1_weights": dense1.weights,
            "dense1_biases": dense1.biases,
            "dense2_weights": dense2.weights,
            "dense2_biases": dense2.biases,
            "dense3_weights": dense3.weights,
            "dense3_biases": dense3.biases,
            "dense4_weights": dense4.weights,
            "dense4_biases": dense4.biases
        }

        with open(filename, 'wb') as file:
            pickle.dump(weghts_and_biases, file)

        print("Saved to pickle file")

    def loadfromPickle(self, filename):
        with open(filename, 'rb') as file:
            weights_and_biases = pickle.load(file)

        dense1 = DenseLayer(28*28, 32)
        dense1.weights = weights_and_biases["dense1_weights"]
        dense1.biases = weights_and_biases["dense1_biases"]

        dense2 = DenseLayer(32, 32)
        dense2.weights = weights_and_biases["dense2_weights"]
        dense2.biases = weights_and_biases["dense2_biases"]

        dense3 = DenseLayer(32, 32)
        dense3.weights = weights_and_biases["dense3_weights"]
        dense3.biases = weights_and_biases["dense3_biases"]

        dense4 = DenseLayer(32, 10)
        dense4.weights = weights_and_biases["dense4_weights"]
        dense4.biases = weights_and_biases["dense4_biases"]

        return dense1, dense2, dense3, dense4


## Get train and test

In [34]:
transform = transforms.ToTensor()

train_dataset = datasets.FashionMNIST(root='./FMNIST', train=True, download=True, transform=transform)

test_dataset = datasets.FashionMNIST(root='./FMNIST', train=False, download=True, transform=transform)

## Initialization

In [43]:
# four layers
loaders = Loaders()

input_dimensions = 28*28
output_dimensions = 10
numOfNeurons = 32

epochs = 5
learnRate = 0.005

dense1 = DenseLayer(input_dimensions, numOfNeurons)
activation1 = RELU()
dense2 = DenseLayer(numOfNeurons, numOfNeurons)
activation2 = RELU()
dense3 = DenseLayer(numOfNeurons, numOfNeurons)
activation3 = RELU()
dense4 = DenseLayer(numOfNeurons, output_dimensions)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

## Train

In [44]:
loaders.train_data(epochs, learnRate, input_dimensions, output_dimensions, train_loader, dense1, activation1, dense2, activation2, dense3, activation3, dense4)

Epoch: 0, Loss: 7.390249121266374
Epoch: 1, Loss: 7.374932262318621
Epoch: 2, Loss: 7.383546259834754
Epoch: 3, Loss: 7.362393289456287
Epoch: 4, Loss: 7.380455976619778


## Test

In [45]:
loaders.test_data(test_loader, dense1, activation1, dense2, activation2, dense3, activation3, dense4)

Accuracy: 10.000000


### Save to pickle

In [46]:
filename = "1905019.pkl"
loaders.savetoPickle(dense1, dense2, dense3, dense4, filename)

Saved to pickle file


### Load pickle

In [47]:
loaders.loadfromPickle(filename)

(<__main__.DenseLayer at 0x7ff4e4b1a010>,
 <__main__.DenseLayer at 0x7ff4d84f9a50>)